In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## NEURALNETWORKSANDDEEPLEARNING/1 MODELPERFORMANCEANDFIT/NEURALNETWORKSANDDEEPLEARNING MODELPERFORMANCEANDFIT 1 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 12: Loading packages  ####

# Helper packages.
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt                     
import numpy as np
import math
import seaborn as sns

# Scikit-learn packages.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
# TensorFlow and supporting packages.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense





In [ ]:
#=================================================-
#### Slide 13: Directory settings  ####

from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 14: Load the data  ####

credit_card = pd.read_csv(str(data_dir) + "/credit_card_data.csv")
print(credit_card.head())




In [ ]:
#=================================================-
#### Slide 17: Data prep: convenience function (cont'd)  ####

def data_prep(df):
    
    # Fill missing values with mean 
    df = df.fillna(df.mean()['BILL_AMT1'])

    # Drop an unnecessary identifier column.
    df = df.drop('ID',axis = 1)

    # Convert 'sex' into dummy variables.
    sex = pd.get_dummies(df['SEX'], prefix = 'sex', drop_first = True)
    # Convert 'education' into dummy variables.
    education = pd.get_dummies(df['EDUCATION'], prefix = 'education', drop_first = True)
    # Convert 'marriage' into dummy variables.
    marriage = pd.get_dummies(df['MARRIAGE'], prefix = 'marriage', drop_first = True)

    # Drop `sex`, `education`, `marriage` from the data.
    df.drop(['SEX', 'EDUCATION', 'MARRIAGE'], axis = 1, inplace = True)

    # Concatenate `sex`, `education`, `marriage` dummies to our dataset.
    df = pd.concat([df, sex, education, marriage], axis=1)
    
    # Separate predictors from data.
    X = df.drop(['default_payment_next_month'], axis=1)

    # Separate target from data.
    y = df['default_payment_next_month']

    # Set the seed to 1.
    np.random.seed(1)
    
    # Split data into train, test, and validation set, use a 70 - 15 - 15 split.
    # First split data into train-test with 70% for train and 30% for test.
    X_train, X_test, y_train, y_test = train_test_split(X.values,
                                                        y,
                                                        test_size = .3,
                                                        random_state = 1)
    
    # Then split the test data into two halves: test and validation. 
    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size = .5,
                                                    random_state = 1)
                                                    
    print("Train shape:", X_train.shape, "Test shape:", X_test.shape, "Val shape:", X_val.shape)
    
    # Transforms features by scaling each feature to a given range.
    # The default is the range between 0 and 1.
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_scaled = min_max_scaler.fit_transform(X_train)
    X_test_scaled = min_max_scaler.transform(X_test)
    X_val_scaled = min_max_scaler.transform(X_val)
    
    return X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val




In [ ]:
#=================================================-
#### Slide 18: Data prep  ####

X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val = data_prep(credit_card)




In [ ]:
#=================================================-
#### Slide 19: Define and compile a sequential model  ####

def create_model(lr = .01):
  # Let's set the seed so that we can reproduce the results.
  tf.random.set_seed(1)
  opt = Adam(learning_rate = lr) # <- set optimizer

  model = Sequential([
          Dense(32, activation='relu', input_dim=30),#<- set input and 1st hidden layer
          Dense(32, activation='relu'),              #<- set 2nd hidden layer
          Dense(1, activation='sigmoid')             #<- set output layer

  ])

  model.compile(optimizer = opt,            #<- set optimizer
                loss='binary_crossentropy', #<- set loss function to binary_crossentropy
                metrics=['accuracy'])       #<- set performance metric
  return model




In [ ]:
#=================================================-
#### Slide 20: Default learning rate  ####

lr_default = create_model().fit(X_train_scaled, y_train,
                                epochs = 25,
                                validation_data=(X_val_scaled,y_val))




In [ ]:
#=================================================-
#### Slide 21: High learning rate  ####

# Set learning rate to 0.75.
lr_high = create_model(lr = .75).fit(X_train_scaled, y_train,  
                                     epochs = 25,            
                                     validation_data=(X_val_scaled, y_val))
                                                




In [ ]:
#=================================================-
#### Slide 22: Low learning rate  ####

lr_low = create_model(lr=.0001).fit(X_train_scaled, y_train,
                                    epochs = 50, #<- increase the number of epochs
                                    validation_data=(X_val_scaled, y_val))
                                                




In [ ]:
#=================================================-
#### Slide 23: Visualize results for learning rates  ####

learn_rates = []

for exp, result in zip([lr_default, lr_low, lr_high], [".01", ".0001", ".75"]):

  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['Learning Rate'] = result

  learn_rates.append(df)

df_learning = pd.concat(learn_rates)
df_learning['Learning Rate'] = df_learning['Learning Rate'].astype('str')




In [ ]:
#=================================================-
#### Slide 24: Visualize results for learning rates (cont'd)  ####

sns.lineplot(x='epoch', y='val_loss', hue='Learning Rate', data=df_learning)




In [ ]:
#=================================================-
#### Slide 25: Visualize results for learning rates (cont'd)  ####

sns.lineplot(x='epoch', y='val_accuracy', hue='Learning Rate', data=df_learning)


#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################


In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## NEURALNETWORKSANDDEEPLEARNING/1 MODELPERFORMANCEANDFIT/NEURALNETWORKSANDDEEPLEARNING MODELPERFORMANCEANDFIT 2 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 4: Define and compile a sequential model  ####

def create_model(lr = .0001):
  # Let's set the seed so that we can reproduce the results.
  tf.random.set_seed(1)
  opt = Adam(learning_rate = lr) # <- set optimizer

  model = Sequential([
          Dense(32, activation='relu', input_dim = 30),#<- set input and 1st hidden layer
          Dense(32, activation='relu'),              #<- set 2nd hidden layer
          Dense(1, activation='sigmoid')             #<- set output layer

  ])

  model.compile(optimizer = opt,            #<- set optimizer
                loss='binary_crossentropy', #<- set loss function to binary_crossentropy
                metrics=['accuracy'])       #<- set performance metric
  return model




In [ ]:
#=================================================-
#### Slide 6: Default batch size  ####

model = create_model()
bt_default = model.fit(X_train_scaled, y_train,                  #<- train data and labels
                       epochs = 25,                              #<- number of epochs
                       validation_data = (X_val_scaled, y_val))  #<- pass val data
                                                




In [ ]:
#=================================================-
#### Slide 7: Small batch size  ####

model = create_model()
bt_small = model.fit(X_train_scaled, y_train,                 #<- train data and labels
                      epochs = 25,                              #<- number of epochs
                      batch_size= 8,                           #<- set batch_size
                      validation_data = (X_val_scaled, y_val))  #<- pass val data
                                                




In [ ]:
#=================================================-
#### Slide 8: Large batch size  ####

model = create_model()
bt_large = model.fit(X_train_scaled, y_train,   #<- train data + labels
                      epochs = 25,              #<- number of epochs
                      batch_size= 512,          #<- set batch_size
                      validation_data = (X_val_scaled, y_val))  #<- val data + labels
                                                




In [ ]:
#=================================================-
#### Slide 9: Visualize results for various batch sizes  ####

batch_sizes = []

for exp, result in zip([bt_default, bt_small, bt_large], ["32", "8", "512"]):

  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['Batch Size'] = result

  batch_sizes.append(df)

df_summary = pd.concat(batch_sizes)
df_summary['Batch Size'] = df_summary['Batch Size'].astype('str')




In [ ]:
#=================================================-
#### Slide 10: Visualize results for various batch sizes (cont'd)  ####


sns.lineplot(x='epoch', y='val_accuracy', 
             hue='Batch Size', data=df_summary)




In [ ]:
#=================================================-
#### Slide 11: Visualize results for various batch sizes (cont'd)  ####

sns.lineplot(x='epoch', y='val_loss', 
             hue='Batch Size', data=df_summary)




In [ ]:
#=================================================-
#### Slide 14: Higher number of epochs  ####

ep_high = create_model().fit(X_train_scaled, y_train,
                             epochs = 150,
                             validation_data=(X_val_scaled, y_val))
                                                




In [ ]:
#=================================================-
#### Slide 15: Medium number of epochs  ####

ep_medium = create_model().fit(X_train_scaled, y_train,
                               epochs = 100,
                               validation_data=(X_val_scaled, y_val))
                                                




In [ ]:
#=================================================-
#### Slide 16: Lower number of epochs  ####

ep_low = create_model().fit(X_train_scaled, y_train,
                            epochs = 25,
                            validation_data = (X_val_scaled, y_val))
                                                




In [ ]:
#=================================================-
#### Slide 17: Visualize results for epoch sizes  ####

epoch_sizes = []

for exp, result in zip([ep_high, ep_medium, ep_low], ["150", "100", "25"]):

  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['Number of epochs'] = result

  epoch_sizes.append(df)

df_epochs = pd.concat(epoch_sizes)
df_epochs['Number of epochs'] = df_epochs['Number of epochs'].astype('str')




In [ ]:
#=================================================-
#### Slide 18: Visualize results for epoch sizes (cont'd)  ####

sns.lineplot(x='epoch', y='val_accuracy', hue='Number of epochs', data=df_epochs)




In [ ]:
#=================================================-
#### Slide 19: Visualize results for epoch sizes (cont'd)  ####

sns.lineplot(x='epoch', y='val_loss', hue='Number of epochs', data=df_epochs)




In [ ]:
#=================================================-
#### Slide 21: Exercise  ####




#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################


In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## NEURALNETWORKSANDDEEPLEARNING/1 MODELPERFORMANCEANDFIT/NEURALNETWORKSANDDEEPLEARNING MODELPERFORMANCEANDFIT 3 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 17: Generate some fake data  ####

TRUE_W = 3.5         #<- true weight
TRUE_b = 50.0        #<- true bias
NUM_EXAMPLES = 1000  #<- number of observations

# Simulate inputs and noise from normal distribution.
inputs = tf.random.normal(shape=[NUM_EXAMPLES])
noise = tf.random.normal(shape=[NUM_EXAMPLES])

# Compute the outputs based on our equation.
outputs = inputs * TRUE_W + TRUE_b + noise




In [ ]:
#=================================================-
#### Slide 18: Neural network architecture  ####

# Define model.
class Model(object):

  def __init__(self):
    self.W = tf.Variable(8.0)   #<- initial weight
    self.b = tf.Variable(40.0)  #<- initial bias

  def __call__(self, x):
    return self.W * x + self.b #<- compute the equation
    
# Initialize the model.
model = Model()

# Check if it outputs correct results.
assert model(3.0).numpy() == 64.0





In [ ]:
#=================================================-
#### Slide 19: Loss function  ####

# Define loss function.
def loss(target_y, predicted_y):
  "MSE"
  return tf.reduce_mean(tf.square(target_y - predicted_y))





In [ ]:
#=================================================-
#### Slide 20: Initial weights  ####

print('Current loss: %1.6f' % loss(model(inputs), 
       outputs).numpy())
plt.scatter(inputs, outputs, c = 'b')
plt.scatter(inputs, model(inputs), c = 'r')
plt.show()




In [ ]:
#=================================================-
#### Slide 21: Update weights based on gradient  ####

# Define the train function for our NN.
def train(model, inputs, outputs, learning_rate):

  with tf.GradientTape() as t: 
     current_loss = loss(outputs, model(inputs)) #<- compute loss
  
  # Compute partial derivatives:
  # how much does a particular obvs + W + b contribute to that loss.
  dW, db = t.gradient(current_loss, [model.W, model.b]) 
  
  # Update with new weights and bias using our learning rate.
  model.W.assign_sub(learning_rate * dW) 
  model.b.assign_sub(learning_rate * db)





In [ ]:
#=================================================-
#### Slide 22: Train the neural network  ####

model = Model()

# Store some history of weights.
Ws, bs = [], []
epochs = range(15)

for epoch in epochs:
  Ws.append(model.W.numpy())
  bs.append(model.b.numpy())
  current_loss = loss(outputs, model(inputs))

  train(model, inputs, outputs, learning_rate=0.1)
  print('Epoch %2d: W=%1.2f b=%1.2f loss=%2.5f' % (epoch, Ws[-1], bs[-1], current_loss))




In [ ]:
#=================================================-
#### Slide 23: Inspect the results  ####

plt.plot(epochs, Ws, 'r', epochs, bs, 'b')
plt.plot([TRUE_W] * len(epochs), 'r--',
         [TRUE_b] * len(epochs), 'b--')
plt.legend(['W', 'b', 'True W', 'True b'])
plt.show()




In [ ]:
#=================================================-
#### Slide 24: Inspect the results (cont'd)  ####

plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()




In [ ]:
#=================================================-
#### Slide 25: Inspect the results (cont'd)  ####

print('Current loss: %1.6f' % loss(model(inputs), outputs).numpy())



#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################
